In [5]:
import pandas as pd
import os

class Plant:
    def __init__(self, image_name, organ, species_id, obs_id, license, partner, gbif_species_id, species, genus, family, learn_tag, image_path=None):
        self.image_name = image_name
        self.organ = organ
        self.species_id = species_id
        self.obs_id = obs_id
        self.license = license
        self.partner = partner
        self.gbif_species_id = gbif_species_id
        self.species = species
        self.genus = genus
        self.family = family
        self.learn_tag = learn_tag
        self.image_path = image_path  # New attribute to store the path of the image


# Assuming the Excel file is named 'plant_data.xlsx' and is in the current directory
file_path = '/home/aycaaktas/PURE/PlantCLEF2024singleplanttrainingdata.csv'
df = pd.read_csv(file_path, delimiter=';')

  


# Directory where images are stored
image_directory_path = '/home/aycaaktas/PURE/DATA/images_max_side_800'

# List to store Plant objects
plants = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    species_folder_path = os.path.join(image_directory_path, str(row['species_id']))
    image_file_path = os.path.join(species_folder_path, row['image_name'])
    
    # Check if the image file exists in the specified path
    if os.path.exists(image_file_path):
        plant = Plant(
            image_name=row['image_name'],
            organ=row['organ'],
            species_id=row['species_id'],
            obs_id=row['obs_id'],
            license=row['license'],
            partner=row['partner'],
            gbif_species_id=row['gbif_species_id'],
            species=row['species'],
            genus=row['genus'],
            family=row['family'],
            learn_tag=row['learn_tag'],
            image_path=image_file_path  # Storing the image path
        )
        plants.append(plant)
    else:
        # You might want to handle cases where the image doesn't exist
        print(f"Image {row['image_name']} not found in {species_folder_path}")

# Example: Print out details of the first few Plant objects to verify
for plant in plants[:5]:  # Just as an example, print details of first 5 plants
    print(f"Species ID: {plant.species_id}, Image: {plant.image_name}, Image Path: {plant.image_path}")


/tmp/ipykernel_474665/2744062047.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=';')


Species ID: 1396710, Image: 59feabe1c98f06e7f819f73c8246bd8f1a89556b.jpg, Image Path: /home/aycaaktas/PURE/DATA/images_max_side_800/1396710/59feabe1c98f06e7f819f73c8246bd8f1a89556b.jpg
Species ID: 1396710, Image: dc273995a89827437d447f29a52ccac86f65476e.jpg, Image Path: /home/aycaaktas/PURE/DATA/images_max_side_800/1396710/dc273995a89827437d447f29a52ccac86f65476e.jpg
Species ID: 1396710, Image: 416235e7023a4bd1513edf036b6097efc693a304.jpg, Image Path: /home/aycaaktas/PURE/DATA/images_max_side_800/1396710/416235e7023a4bd1513edf036b6097efc693a304.jpg
Species ID: 1396710, Image: cbd18fade82c46a5c725f1f3d982174895158afc.jpg, Image Path: /home/aycaaktas/PURE/DATA/images_max_side_800/1396710/cbd18fade82c46a5c725f1f3d982174895158afc.jpg
Species ID: 1396710, Image: f82c8c6d570287ebed8407cefcfcb2a51eaaf56e.jpg, Image Path: /home/aycaaktas/PURE/DATA/images_max_side_800/1396710/f82c8c6d570287ebed8407cefcfcb2a51eaaf56e.jpg


In [6]:
import torch
print(torch.cuda.is_available())  # This should print True if CUDA is properly set up
print(torch.cuda.get_device_name(0))  # This should print the name of your GPU if available


True
NVIDIA RTX A6000


In [19]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms.functional as F
from PIL import Image, ImageDraw
from torchvision.ops import nms



# Modify the Faster R-CNN model
def load_modified_model(num_classes=2):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    for name, parameter in model.named_parameters():
        if 'box_predictor' not in name:
            parameter.requires_grad = False
    return model

model = load_modified_model()

# Function to segment and label images based on green color
def segment_and_label_image(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 52, 72])
    upper_green = np.array([102, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    green_area = np.sum(mask > 0)
    total_area = mask.size
    label = 1 if (green_area / total_area) > 0.1 else 0
    print(f"Processed an image segment - Label: {label}, Green Area: {green_area}, Total Area: {total_area}")
    return mask, label

# Split image into three parts and generate training data
def prepare_data(plant_objects, limit=None):
    training_data = []
    count = 0
    for plant in plant_objects:
        if limit and count >= limit:
            break
        image = cv2.imread(plant.image_path)
        if image is None:
            print(f"Failed to load image at {plant.image_path}")
            continue
        height, width, _ = image.shape
        segments = [
            (image[:, :width//3], [0, 0, width//3, height]),
            (image[:, width//3:2*width//3], [width//3, 0, 2*width//3, height]),
            (image[:, 2*width//3:], [2*width//3, 0, width, height])
        ]
        for segment, bbox in segments:
            mask, label = segment_and_label_image(segment)
            segment_tensor = F.to_tensor(Image.fromarray(segment))
            bbox_tensor = torch.tensor([bbox], dtype=torch.float32)
            label_tensor = torch.tensor([label], dtype=torch.int64)
            training_data.append((segment_tensor, {'boxes': bbox_tensor, 'labels': label_tensor}))
        count += 1
    return training_data


# Continue to use this modified function in your training pipeline







# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]



# Training function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_model(model, data_loader):
    print("Starting the training...")
    model.to(device)
    model.train()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.005)
    for images, targets in data_loader:
        images = [image.to(device) for image in images]
        # Check and format targets correctly
        formatted_targets = []
        for t in targets:
            if isinstance(t, dict):
                formatted_target = {k: v.to(device) for k, v in t.items()}
                formatted_targets.append(formatted_target)
        optimizer.zero_grad()
        if formatted_targets:
            loss_dict = model(images, formatted_targets)
            losses = sum(loss for loss in loss_dict.values())
            losses.backward()
            optimizer.step()
            print(f"Loss: {losses.item()}")
    print("Training finished.")
    return model






def visualize_some_predictions(model, image_paths, number=5, output_dir="output_images"):
    model.eval()
    device = next(model.parameters()).device
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    for i, image_path in enumerate(image_paths):
        if i >= number:
            break
        image = Image.open(image_path).convert("RGB")
        image_tensor = F.to_tensor(image).to(device)

        with torch.no_grad():
            predictions = model([image_tensor])

        if predictions[0]['boxes'].shape[0] == 0:
            print("No boxes detected for image:", image_path)
            continue

        draw = ImageDraw.Draw(image)
        for element in predictions[0]['boxes']:
            boxes = element.cpu().numpy()
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], outline="green", width=3)
        
        output_image_path = os.path.join(output_dir, f"output_{i}.png")
        image.save(output_image_path)
        print(f"Saved visualized prediction to {output_image_path}")


def apply_nms(orig_predictions, iou_thresh=0.3, score_thresh=0.5):
    # Apply NMS on predictions
    keep_boxes = nms(orig_predictions['boxes'], orig_predictions['scores'], iou_thresh)

    final_predictions = {
        'boxes': orig_predictions['boxes'][keep_boxes],
        'scores': orig_predictions['scores'][keep_boxes],
        'labels': orig_predictions['labels'][keep_boxes],
    }

    # Filter out low scoring boxes
    keep_scores = final_predictions['scores'] > score_thresh
    final_predictions = {
        'boxes': final_predictions['boxes'][keep_scores],
        'scores': final_predictions['scores'][keep_scores],
        'labels': final_predictions['labels'][keep_scores],
    }
    
    return final_predictions

def visualize_predictions_with_nms(model, image_path, output_dir="output_images"):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    model.eval()
    device = next(model.parameters()).device
    image = Image.open(image_path).convert("RGB")
    image_tensor = F.to_tensor(image).to(device)
    
    with torch.no_grad():
        predictions = model([image_tensor])[0]
    
    # Apply non-maximum suppression with confidence thresholding
    nms_predictions = apply_nms(predictions, iou_thresh=0.3, score_thresh=0.5)

    draw = ImageDraw.Draw(image)
    for box in nms_predictions['boxes']:
        boxes = box.cpu().numpy()
        draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], outline="red", width=3)

    # Save the image with drawn boxes to the output directory
    output_image_path = os.path.join(output_dir, os.path.basename(image_path))
    image.save(output_image_path)
    print(f"Saved visualized prediction to {output_image_path}")

# Example usage

# Adjust prepare_data and train_model to handle just one image
print("Before training...")
# Assuming this function is correctly setting up your data_loader
training_data = prepare_data(plants, limit=5)  # Test with one item to reduce load
dataset = CustomDataset(training_data)
data_loader = DataLoader(dataset, batch_size=1)  # Using a small batch size for simplicity

for images, targets in data_loader:
    # Process each batch here
    print(images, targets)  # Example processing

trained_model = train_model(model, data_loader)

# Try to visualize the result of just one image
test_image_paths = [plants[0].image_path]  # Use the first plant's image
visualize_some_predictions(trained_model, test_image_paths, number=5)
visualize_predictions_with_nms(trained_model, '/home/aycaaktas/PURE/output_images/output_0.png')





/home/aycaaktas/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/aycaaktas/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Before training...
Processed an image segment - Label: 1, Green Area: 84245, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 94245, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 88021, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 96096, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 102054, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 73737, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 24769, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 28919, Total Area: 160000
Processed an image segment - Label: 0, Green Area: 14380, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 48429, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 47675, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 52279, Total Area: 160000
Processed an image segment - Label: 1, Green Area: 53878